In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [38]:
g = torch.Generator().manual_seed(1337)
vocab_size = 65
emb_size = 32
head_size = 4
block_size = 12
batch_size = 10
layer_size = 4
tril = torch.ones(block_size, block_size)

max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):

    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class head(nn.Module):
    def __init__(self):
        super().__init__()
        self.key = nn.Linear(emb_size, emb_size//4, bias=False)
        self.query = nn.Linear(emb_size, emb_size//4, bias=False)
        self.value = nn.Linear(emb_size, emb_size//4, bias=False)

    def forward(self, x):
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        wei1 = q @ k.transpose(-2, -1)
        wei1 = wei1.masked_fill(tril==0, float('-inf'))
        wei1 = F.softmax(wei1, dim=-1)
        out = wei1 @ v
        return out

class multihead(nn.Module):
    def __init__(self):
        super().__init__()
        self.attention1 = head()
        self.attention2 = head()
        self.attention3 = head()
        self.attention4 = head()
        self.linear = nn.Linear(emb_size, emb_size)
        self.project = nn.Linear(emb_size, emb_size)
        
    def forward(self, x):
        out1 = self.attention1(x)
        out2 = self.attention2(x)
        out3 = self.attention3(x)
        out4 = self.attention4(x)
        out = torch.cat([out1, out2, out3, out4], dim=-1)
        out = self.project(out)
        out = self.linear(out)
        return out

class feedforward(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(emb_size, 4*emb_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(4*emb_size, emb_size)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x

class Block(nn.Module):
    def __init__(self):
        super().__init__()
        self.block = multihead()
        self.ffd = feedforward()
        self.ln1 = nn.LayerNorm(emb_size)
        self.ln2 = nn.LayerNorm(emb_size)

    def forward(self, x):
        x = x + self.block(self.ln1(x))              # Residual Connection
        x = x + self.ffd(self.ln2(x))                # Residual Connection
        return x

class LanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, emb_size)
        self.position_emb = nn.Embedding(block_size, emb_size)
        self.block1 = Block()
        self.block2 = Block()
        self.block3 = Block()
        self.block4 = Block()
        self.ln = nn.LayerNorm(emb_size)
        self.layer = nn.Linear(emb_size, vocab_size)

    def forward(self, x, targets=None):
        x = self.token_emb(x) + self.position_emb(torch.arange(block_size))
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.ln(x)
        logits = self.layer(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss


In [ ]:
model = LanguageModel()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.4311, val loss 4.4313
step 100: train loss 3.0415, val loss 3.0605
step 200: train loss 2.0805, val loss 2.1423
step 300: train loss 0.9245, val loss 0.9611
step 400: train loss 0.5571, val loss 0.5998
step 500: train loss 0.4193, val loss 0.4345
step 600: train loss 0.3498, val loss 0.3490
step 700: train loss 0.2976, val loss 0.2975
step 800: train loss 0.2745, val loss 0.2818
step 900: train loss 0.2580, val loss 0.2516
step 1000: train loss 0.2519, val loss 0.2557
step 1100: train loss 0.2433, val loss 0.2432
step 1200: train loss 0.2341, val loss 0.2383
step 1300: train loss 0.2322, val loss 0.2354
step 1400: train loss 0.2306, val loss 0.2340
step 1500: train loss 0.2224, val loss 0.2232
step 1600: train loss 0.2220, val loss 0.2221
step 1700: train loss 0.2175, val loss 0.2273
step 1800: train loss 0.2246, val loss 0.2175
step 1900: train loss 0.2180, val loss 0.2179
step 2000: train loss 0.2082, val loss 0.2083
step 2100: train loss 0.2149, val loss 0.2201
